In [49]:
import os
import numpy as np
from PIL import Image
from skimage.io import imread
import tensorflow as tf
import tensorflow_datasets as tfds
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from keras import backend as K

In [50]:
test_path = "../input/cattailnotcattail/Cattails Dataset/Test"
train_path = "../input/cattailnotcattail/Cattails Dataset/Train"

In [51]:
# Creating a list of labels
labels = os.listdir(train_path)

# Viewing the labels
labels

In [52]:
# get all the data in the directory split/train and reshape them
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_path, 
        target_size=(227, 227), batch_size=500)

# get all the data in the directory split/test and reshape them
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_path, 
        target_size=(227, 227), batch_size =500) 

In [53]:
# Assigning image and labels to the variables

train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)

In [54]:
# Splitting a validation dataset from the train dataset

train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, 
                                                                      random_state=15)

In [55]:
# Checking a sample image and its label

display(plt.imshow(train_images[5]))
print(train_labels[5])

In [56]:
# Setting up the Augmentation
datagen = ImageDataGenerator(
        rotation_range=30,
        horizontal_flip=True,
        fill_mode='nearest')

# fit parameters from train_images
datagen.fit(train_images, augment=True)

In [57]:
base_model = VGG16(include_top=False, input_shape = (227, 227, 3), weights = 'imagenet')

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(2,activation='sigmoid'))
model.summary()

In [58]:
# Compiling the model
model.compile(optimizer="sgd", loss="binary_crossentropy", metrics=["accuracy"])

# Setting the learning rate as 0.0015
K.set_value(model.optimizer.learning_rate, 0.0005)

In [59]:
results = model.fit(datagen.flow(train_images, train_labels, batch_size=32), epochs = 20, 
                        validation_data =(val_images, val_labels), steps_per_epoch=len(train_images) / 32)

In [60]:
# Checking model's performance with training data
results_train = model.evaluate(train_images, train_labels)

In [61]:
# Checking model's performance with testing data
results_test = model.evaluate(test_images, test_labels)

In [62]:
def download_and_predict(url, filename):
    
    # download the image from the url and save
    os.system("curl -s {} -o {}".format(url, filename))
    
    # open the image
    img = Image.open(filename)
    
    # save the image
    img.save(filename)
    
    # convert it to RGB
    img = img.convert('RGB')
    
    # show image
    plt.imshow(img)
    plt.axis('off')
    
    # resize the image for VGG16 model
    img = img.resize((227, 227))
    
    # save the image
    img.save(filename)
        
    # calculate probabilities of breeds
    img = imread(filename)
    img = preprocess_input(img)
    probs = model.predict(np.expand_dims(img, axis=0))
    pred = probs.argsort()[0][::-1][0]
    
    if pred == 1.:
        print("It's a Cattail!")
    else:
        print("It's not a Cattail!")   

In [73]:
download_and_predict("https://cdn4.eyeem.com/thumb/739cd4a342f84873793e3492da23d91d44df09c7-1576425116352/w/800", "test_1.jpg")

In [64]:
download_and_predict("https://cdn4.eyeem.com/thumb/d7768b0cfac2b6e0d4e122cc1d7e669e5b2ee230-1469222679472/w/800", "test_2.jpg")

In [69]:
download_and_predict("https://lh3.googleusercontent.com/r_T5eZtvPXrSWVok1ViGZGgAkR4U1sePYOG7ajPpHlGGBzHEhICQHE2rwoRb1qJErYowfEj19pyMHFMWgZXS5w=s900", "test_3.jpg")

In [66]:
download_and_predict("https://img.myloview.com/stickers/green-cactus-plant-with-white-thorn-nature-green-desert-plant-backdrop-and-beautiful-detail-700-265659688.jpg", "test_4.jpg")

In [74]:
download_and_predict("https://i.ytimg.com/vi/imi8YDvyWn0/maxresdefault.jpg", "test_4.jpg")